In [1]:
import pandas as pd
import tqdm

import re
import requests
from bs4 import BeautifulSoup

import hashlib
import datetime

In [2]:
base_url = 'http://www.curitiba.pr.gov.br/dadosabertos/consulta'

In [3]:
params = {"grupo":"13"}

In [4]:
r = requests.get(base_url, params=params)

In [5]:
page = BeautifulSoup(r.text, 'html.parser')

In [6]:
__EVENTVALIDATION = page.find("input", {'name': '__EVENTVALIDATION'}).get('value')

In [7]:
__EVENTVALIDATION

'/wEdAAwIu/PP9rhmFtVJtY8JFGFw4mKL5o1QRo/FpCKPmYsWJJ6M545mi+eVxYOsufjJuPs5znhzSWYfXU4ugll/AJGuS7DlCF7QoIFOF8dSWE2wXzkiLQS8NWTPNZDV80Z3AzzphndoyDFnIVED7A3KQw9cs48DT8q4XVH83r8Yw95EbJ8/8wAqn890Z+wGGuGKtYqNvjaF5GeBQ6wPsZcNPGG+vCOATgFUUjl7PbiZupqgZarqqZvleoeig3SW/qIZQc3bEKA53vmAOV75/LIJc13EmUvGzw=='

In [8]:
__VIEWSTATE = page.find("input", {'name': '__VIEWSTATE'}).get('value')

In [9]:
__VIEWSTATE

'/wEPDwUJMzc0Mzc1MzA2DxYGHgtzdGF0ZV9ncnVwbwUCMTMeCGlkZkdydXBvAg0eCGJ1c2NhUG9yZBYCZg9kFgICAw9kFgICAQ8WAh4GYWN0aW9uBR8vZGFkb3NhYmVydG9zL2NvbnN1bHRhP2dydXBvPTEzFg4CAg8QDxYGHg1EYXRhVGV4dEZpZWxkBQpOb21lUGFnaW5hHg5EYXRhVmFsdWVGaWVsZAUJVXJsUGFnaW5hHgtfIURhdGFCb3VuZGcWBB4Jb25rZXlkb3duBUNpZihJc0VudGVyS2V5KGV2ZW50KSl7UmVkaXJlY2lvbmFyQWNlc3NvUmFwaWRvKHRoaXMpO3JldHVybiBmYWxzZTt9HgdvbmNsaWNrBR9SZWRpcmVjaW9uYXJBY2Vzc29SYXBpZG8odGhpcyk7EBUGDkFjZXNzbyBSw6FwaWRvFlNlY3JldGFyaWFzIGUgw5NyZ8Ojb3MRU2VydmnDp29zIE9uLUxpbmUYU2VydmnDp29zIHBhcmEgYSBFbXByZXNhGVNlcnZpw6dvcyBwYXJhIG8gQ2lkYWTDo28TU2l0ZXMgZGEgUHJlZmVpdHVyYRUGAA0vc2VjcmV0YXJpYXMvGS9saXN0YS1zZXJ2aWNvcy1vbi1saW5lLzIRL3NlcnZpY29zL2VtcHJlc2ERL3NlcnZpY29zL2NpZGFkYW8cL2xpc3RhLXNpdGVzLWRhLXByZWZlaXR1cmEvNRQrAwZnZ2dnZ2dkZAIEDw9kFgIfBwVJcmV0dXJuIFZhbGlkYXJQZXNxdWlzYSgndHh0UGVzcXVpc2FUb3BvJywnY3RsMDAkbG5rUGVzcXVpc2FUb3BvJywgZXZlbnQpO2QCBg8PFgIeBFRleHQFDlPDoWIsIDEzLzEwLzE4ZGQCBw8PFgIeCEltYWdlVXJsBRV+L2ltYWdlbS90ZW1wby9wYy5naWZkZAIIDw8WAh8JBQQyNcK6ZGQCCQ8PFgIfCQU

In [10]:
pagination = page.find_all(class_='paginacao2')
pagination = list(pagination[0].text.replace('\n', ''))

In [11]:
pages = ""
for page in pagination:
    data = {
        "__EVENTTARGET": f"ctl00$cphMasterPortal$ucPaginador${page}_pg{page}",
        "__EVENTVALIDATION": __EVENTVALIDATION,
        "__VIEWSTATE": __VIEWSTATE
    }
    
    r = requests.request("POST", base_url, params=params, data=data)
    pages = pages + r.text

In [12]:
md5 = lambda s : hashlib.md5(s.encode('utf-8')).hexdigest()

In [13]:
md5('test')

'098f6bcd4621d373cade4e832627b4f6'

In [14]:
a_tags = BeautifulSoup(pages, 'html.parser').find_all(id='linkArquivo')

In [15]:
print(md5(a_tags[0].text))

58523cf9d504bccd5544356dbf6ca665


In [16]:
date_regex = re.compile('\d{4}-\d{2}-\d{2}')

In [17]:
date_string = date_regex.search('//multimidia.curitiba.pr.gov.br/dadosabertos/SESPAMedicoUnidadesMunicipaisDeSaude/2018-10-10_Sistema_E-Saude_Medicos_-_Base_de_Dados.csv').group()
date = datetime.date(*map(int, date_string.split('-')))
date


datetime.date(2018, 10, 10)

In [18]:
databases = []
for a_tag in tqdm.tqdm(a_tags):
    database_link = a_tag.get('href')[2:]
    
    if database_link[-4:] == '.csv':
        database_id = md5(a_tag.text)
        database_title = a_tag.text

        database_date_string = date_regex.search(database_link).group()
        database_date = datetime.date(*map(int, database_date_string.split('-')))

        database = {
            'database_id': database_id,
            'database_title': database_title,
            'database_link': database_link,
            'database_date': database_date
        }
        
        databases.append(database)

100%|██████████| 7/7 [00:00<00:00, 5372.39it/s]


In [19]:
databases

[{'database_id': '58523cf9d504bccd5544356dbf6ca665',
  'database_title': 'imapaprendere - Base de Dados',
  'database_link': 'multimidia.curitiba.pr.gov.br/dadosabertos/IMAPAprendere/2018-10-01_imapaprendere_-_Base_de_Dados.csv',
  'database_date': datetime.date(2018, 10, 1)}]

In [21]:
test = pd.read_csv('http://multimidia.curitiba.pr.gov.br/dadosabertos/IMAPAprendere/2018-10-01_imapaprendere_-_Base_de_Dados.csv', encoding="ISO-8859-1", delimiter=';', error_bad_lines=False)

b'Skipping line 5201: expected 7 fields, saw 8\nSkipping line 7664: expected 7 fields, saw 8\nSkipping line 8502: expected 7 fields, saw 8\nSkipping line 9955: expected 7 fields, saw 8\nSkipping line 9960: expected 7 fields, saw 8\nSkipping line 10060: expected 7 fields, saw 8\nSkipping line 10085: expected 7 fields, saw 8\nSkipping line 10128: expected 7 fields, saw 8\nSkipping line 10199: expected 7 fields, saw 8\n'


In [22]:
test.fillna('').head()

,ORGAO,CURSO,ANO_EXECUCAO,AREA_TEMATICA,CARGA_HORARIA,TOTAL_INSCRITOS,TOTAL_CERTIFICADOS
0,------,-----,------------,-------------,-------------,---------------,------------------
1,INSTITUTO MUNICIPAL DE ADMINISTRAÇÃO PÚBLICA,5S - A base para a qualidade,2008,GESTÃO DE PESSOAS,4,2118,1717
2,INSTITUTO MUNICIPAL DE ADMINISTRAÇÃO PÚBLICA,A Arte da Guerra. Estratégias de Sucesso para ...,2008,TECNOLOGIA DA INFORMAÇÃO E DA COMUNICAÇÃO,4,78,54
3,INSTITUTO MUNICIPAL DE ADMINISTRAÇÃO PÚBLICA,A arte de falar em público,2008,GESTÃO DE PESSOAS,8,202,122
4,INSTITUTO MUNICIPAL DE ADMINISTRAÇÃO PÚBLICA,A AVALIAÇÃO NA GESTÃO EM REDE DE PROJETOS SOCIAIS,2008,INOVAÇÃO SOCIAL,4,142,140
